# FewShotPromptTemplate

## 1. Set API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. FewShotTemplate

다음과 같은 방식으로 질문한다고 했을 때,
- 질문이 복잡해 지거나,
- 답변의 형식을 지정하고 싶은 경우

올바른 답변을 보장할 수 없음

> 이럴 떄, one-shot, few-shot 과 같은 prompting 기술을 사용해야함.
>> one-shot? few-shot? : 답변의 예시를 보여주는 것

- one-shot : 1 건의 답변 예시만을 보여주는 것
- few-shot : 2 건 이상의 여러 답변 예시는 보여주는 것
- zero-shot : 답변 예시를 제공하지 않는 것

GPT는 답변 예시를 따라하는 것을 잘 수행함, zero-shot의 경우 질문자의 의도를 정확히 파악해 그것에 맞는 답변을 해주기 어려움

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

question = "대한민국의 수도는 ?"

print(llm.invoke(question).content)

대한민국의 수도는 서울입니다.


아래와 같이 LLM이 한번에 답변하기 어려운 내용애 대해
- 일련의 논리적인 추론 과정을 보여주고
- 이를 토대로 답변할 수 있도록 만들어주는 것이 더 좋은 답변을 기대할 수 있음
    - 즉, 답변 예시를 제공하는 것이 "Hallucination"을 줄여주며, 더 정확한 답변을 얻는데에도 도움이 됨

In [5]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
        """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
        """,
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 율곡 이이의 어머니는 누구인가요?
            중간 답변: 율곡 이이의 어머니는 신사임당입니다.
            추가 질문: 신사임당은 언제 태어났나요?
            중간 답변: 신사임당은 1504년에 태어났습니다.
            추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
            중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
            최종 답변은: 연산군
        """,
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 올드보이의 감독은 누구인가요?
            중간 답변: 올드보이의 감독은 박찬욱입니다.
            추가 질문: 박찬욱은 어느 나라 출신인가요?
            중간 답변: 박찬욱은 대한민국 출신입니다.
            추가 질문: 기생충의 감독은 누구인가요?
            중간 답변: 기생충의 감독은 봉준호입니다.
            추가 질문: 봉준호는 어느 나라 출신인가요?
            중간 답변: 봉준호는 대한민국 출신입니다.
            최종 답변은: 예
        """,
    },
]

In [8]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}\n"
)

print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
        



In [9]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
        


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
        


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 율곡 이이의 어머니는 누구인가요?
            중간 답변: 율곡 이이의 어머니는 신사임당입니다.
            추가 질문: 신사임당은 언제 태어났나요?
            중간 답변: 신사임당은 1504년에 태어났습니다.
            추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
            중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
            최종 답변은: 연산군
        


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 올드보이의 감독은 누구인가요?
            중간 답변

In [10]:
print(llm.invoke(final_prompt).content)

이 질문에 추가 질문이 필요한가요: 예.
            
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.

추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.

추가 질문: 1998년에 Bill Gates는 몇 살이었나요?
중간 답변: 1998년에 Bill Gates는 43세였습니다.

최종 답변은: 43세


In [11]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

chain = prompt | llm

print(chain.invoke(question).content)

이 질문에 추가 질문이 필요한가요: 예.
            
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.

추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.

추가 질문: 1998년에 Bill Gates는 몇 살이었나요?
중간 답변: 1998년에 Bill Gates는 43세였습니다.

최종 답변은: 43세
